In [1]:
import mne, os
import numpy as np
import pandas as pd 
from joblib import Parallel, delayed



In [8]:

#*****************************#
### HELPER FUNCTIONS ###
#*****************************#
def setup_paths(meg_dir, session):
    run_paths,event_paths = [],[]
    for file in os.listdir(f'{meg_dir}/ses-{str(session).zfill(2)}/meg/'):
        if file.endswith(".ds") and file.startswith("sub"):
            run_paths.append(os.path.join(f'{meg_dir}/ses-{str(session).zfill(2)}/meg/', file))
        if file.endswith("events.tsv") and file.startswith("sub"):
            event_paths.append(os.path.join(f'{meg_dir}/ses-{str(session).zfill(2)}/meg/', file))
    run_paths.sort()
    event_paths.sort()

    return run_paths, event_paths 

def read_raw(curr_path,session,run,participant):
    raw = mne.io.read_raw_ctf(curr_path,preload=True)
    # signal dropout in one run -- replacing values with median
    if participant == '1' and session == 11 and run == 4:  
        n_samples_exclude   = int(0.2/(1/raw.info['sfreq']))
        raw._data[:,np.argmin(np.abs(raw.times-13.4)):np.argmin(np.abs(raw.times-13.4))+n_samples_exclude] = np.repeat(np.median(raw._data,axis=1)[np.newaxis,...], n_samples_exclude, axis=0).T
    elif participant == '2' and session == 10 and run == 2: 
        n_samples_exclude = int(0.2/(1/raw.info['sfreq']))
        raw._data[:,np.argmin(np.abs(raw.times-59.8)):np.argmin(np.abs(raw.times-59.8))+n_samples_exclude] = np.repeat(np.median(raw._data,axis=1)[np.newaxis,...], n_samples_exclude, axis=0).T

    raw.drop_channels('MRO11-1609')
        
    return raw

def read_events(event_paths,run,raw):
    # load event file that has the corrected onset times (based on optical sensor and replace in the events variable)
    event_file = pd.read_csv(event_paths[run],sep='\t')
    event_file.value.fillna(999999,inplace=True)
    events = mne.find_events(raw, stim_channel=trigger_channel,initial_event=True)
    events = events[events[:,2]==trigger_amplitude]
    events[:,0] = event_file['sample']
    events[:,2] = event_file['value']
    return events

def concat_epochs(raw, events, epochs):
    if epochs:
        epochs_1 = mne.Epochs(raw, events, tmin = pre_stim_time, tmax = post_stim_time, picks = 'mag',baseline=None)
        epochs_1.info['dev_head_t'] = epochs.info['dev_head_t']
        epochs = mne.concatenate_epochs([epochs,epochs_1])
    else:
        epochs = mne.Epochs(raw, events, tmin = pre_stim_time, tmax = post_stim_time, picks = 'mag',baseline=None)
    return epochs

def baseline_correction(epochs):
    baselined_epochs = mne.baseline.rescale(data=epochs.get_data(),times=epochs.times,baseline=(None,0),mode='zscore',copy=False)
    epochs = mne.EpochsArray(baselined_epochs, epochs.info, epochs.events, epochs.tmin,event_id=epochs.event_id)
    return epochs

def stack_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution):
    for epochs in session_epochs:
        epochs.info['dev_head_t'] = session_epochs[0].info['dev_head_t']
    all_epochs = mne.concatenate_epochs(epochs_list = session_epochs, add_offset=True)
    all_epochs.metadata = pd.read_csv(f'{sourcedata_dir}/sample_attributes_P{str(participant)}.csv')
    all_epochs.decimate(decim=(1200/output_resolution))
    all_epochs.save(f'{preproc_dir}/preprocessed_P{str(participant)}-epo.fif', overwrite=True)
    print(all_epochs.info)

def save_per_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution, sess_list):
    start_id = 0
    for sess, epochs in zip(sess_list, session_epochs):
        end_id = start_id + len(epochs)
        metadata = pd.read_csv(f'{sourcedata_dir}/sample_attributes_P{str(participant)}.csv')
        epochs.metadata = metadata.iloc[start_id:end_id]
        epochs.decimate(decim=(1200/output_resolution))
        savefile = f'{preproc_dir}/preprocessed_P{str(participant)}-epo-sess{sess}.fif'
        epochs.save(savefile, overwrite=True)
        start_id = end_id
    print(savefile)
    
    
#*****************************#
### FUNCTION TO RUN PREPROCESSING ###
#*****************************#
def run_preprocessing(meg_dir,session,participant):
    epochs = []
    run_paths, event_paths = setup_paths(meg_dir, session)
    for run, curr_path in enumerate(run_paths):
        raw = read_raw(curr_path,session,run, participant)
        events = read_events(event_paths,run,raw)
        raw.filter(l_freq=l_freq,h_freq=h_freq)
        epochs = concat_epochs(raw, events, epochs)
        epochs.drop_bad()
    print(epochs.info)
    epochs = baseline_correction(epochs)
    return epochs


In [88]:
#*****************************#
### PARAMETERS ###
#*****************************#

n_sessions                  = 12
trigger_amplitude           = 64
l_freq                      = 0.1
h_freq                      = 40
pre_stim_time               = -0.1
post_stim_time              = 1.3
std_deviations_above_below  = 4
output_resolution           = 200
trigger_channel             = 'UPPT001'



class Args():
        bids_dir = '/work/project/MEG_GOD/yainoue/things/MEG'
        image_dir = '/work/project/MEG_GOD/yainoue/things/Images'
        participant = 1

args = Args()

bids_dir                    = args.bids_dir
participant                 = args.participant
meg_dir                     = f'{bids_dir}/sub-BIGMEG{participant}/'
sourcedata_dir              = f'{bids_dir}/sourcedata/'
preproc_dir                 = f'{bids_dir}/derivatives/preprocessed/'
if not os.path.exists(preproc_dir):
    os.makedirs(preproc_dir)
    
session_epochs = []
for session in range(1, n_sessions+1):
    epoch = run_preprocessing(meg_dir,session,participant)
    session_epochs.append(epoch)

# stack_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution)

ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.92   77.00    0.00 mm <->   -0.92   77.00    0.00 mm (orig :  -40.06   68.57 -237.90 mm) diff =    0.000 mm
       0.92  -77.00    0.00 mm <->    0.92  -77.00    0.00 mm (orig :   70.03  -39.14 -237.69 mm) diff =    0.000 mm
      79.58    0.00    0.00 mm <->   79.58   -0.00    0.00 mm (orig :   71.23   70.87 -233.83 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-01_meg.ds/sub-BIGMEG1_ses-01_task-main_run-01_meg.meg4: 
    System clock channel is available, checking which

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.4s finished


Not setting metadata
226 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 226 events and 1681 original time points ...
0 bad epochs dropped
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.71   77.07    0.00 mm <->   -0.71   77.07    0.00 mm (orig :  -40.27   68.70 -237.97 mm) diff =    0.000 mm
       0.71  -77.07    0.00 mm <->    0.71  -77.07    0.00 mm (orig :   69.91  -39.09 -237.96 mm) diff =    0.000 mm
      79.46    0.00    0.00 mm <->   79.46   -0.00    0.00 mm (orig :   70.83   71.02 -233.95 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinat

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.2s finished


Not setting metadata
226 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 226 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 226 events and 1681 original time points ...
Using data from preloaded Raw for 226 events and 1681 original time points ...
Not setting metadata
452 matching events found
No baseline correction applied
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-03_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.64   77.10    0.00 mm <->   -0.64   77.10   -0.00 mm (orig :  -40.41   68.84 -238.00 mm) diff =    0.000 mm
       0.64  -77.10    0.00 mm <->    0.64  -77.10    0.00 mm (orig :   69.73  -39.08 -238.46 mm) diff

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.4s finished


Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
1804 matching events found
No baseline correction applied
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-09_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.84   77.17    0.00 mm <->   -0.84   77.17    0.00 mm (orig :  -40.16   68.71 -237.52 mm) diff =    0.000 mm
       0.84  -77.17    0.00 mm <->    0.84  -77.17    0.00 mm (orig :   70.00  -39.40 -238.12 mm) diff =    0.000 mm
      79.17    0.00    0.00 mm <->   79.17   -0.00    0.00 mm (

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.4s finished


Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped


KeyboardInterrupt: 

In [9]:
save_per_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution, list(range(1, n_sessions+1)))

Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


Adding metadata with 18 columns


...

Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess11.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess12.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess12.fif


In [11]:
total_n = 0
for epochs in session_epochs:
    print(len(epochs))
    total_n += len(epochs)
print('total: ', total_n)

2254
2254
2254
2254
2254
2254
2254
2254
2254
2254
2254
2254
total:  27048


In [13]:
27048 - 22448

4600

In [37]:
2254 * 12

27048

In [26]:
epoch.metadata.keys()

Index(['trial_type', 'image_nr', 'category_nr', 'exemplar_nr', 'test_image_nr',
       'things_category_nr', 'things_image_nr', 'things_exemplar_nr',
       'image_path', 'onset', 'image_on', 'image_off', 'responded', 'key_id',
       'key_time', 'RT', 'session_nr', 'run_nr'],
      dtype='object')

In [27]:
exp_df = epoch.metadata.query('trial_type=="exp"')
test_df = epoch.metadata.query('trial_type=="test"')

In [39]:
NUM_TRAIN = 0
NUM_TEST = 0
for epoch in session_epochs:
    exp_df = epoch.metadata.query('trial_type=="exp"')
    test_df = epoch.metadata.query('trial_type=="test"')
    catch_df = epoch.metadata.query('trial_type=="catch"') # artificial image
    print('total: ', len(epoch.metadata), 'train: ', len(exp_df), 'test: ', len(test_df), 'catch: ', len(catch_df))
    print('train max', exp_df['image_nr'].max()) # 22248終わり
    print('train min', exp_df['image_nr'].min()) # 1始まり
    print('test max', test_df['image_nr'].max()) # 22648終わり
    print('test min', test_df['image_nr'].min()) # 22449始まり
    print('catch max', catch_df['image_nr'].max()) # 22448 終わり
    print('catch min', catch_df['image_nr'].min()) # 22249始まり
    NUM_TRAIN +=  len(exp_df)
    NUM_TEST += len(test_df)
print('TRAIN: ', NUM_TRAIN)
print('TEST: ', NUM_TEST)

total:  2254 train:  1854 test:  200 catch:  200
train max 22246
train min 10
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22238
train min 2
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22237
train min 1
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22248
train min 12
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22243
train min 7
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22242
train min 6
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22241
train min 5
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854

In [59]:
print(epoch.metadata['trial_type'].unique())

print(epoch.metadata.query('trial_type=="exp"').index.to_numpy())
print(epoch.metadata.query('trial_type=="test"').index)

['exp' 'test' 'catch']
[   0    1    3 ... 2251 2252 2253]
Int64Index([   2,    4,    7,   23,   30,   34,   55,   73,   82,   87,
            ...
            2120, 2139, 2141, 2147, 2153, 2157, 2165, 2216, 2225, 2241],
           dtype='int64', length=200)


In [57]:
epoch.metadata.iloc[[ 0,    1,   2,  3,  4,  5, 6,7,   8,    9]]

,trial_type,image_nr,category_nr,exemplar_nr,test_image_nr,things_category_nr,things_image_nr,things_exemplar_nr,image_path,onset,image_on,image_off,responded,key_id,key_time,RT,session_nr,run_nr
0,exp,3579,299,3,NaN,299.0,4353.0,3.0,images_meg/chain/chain_03s.jpg,3.00,3.006116,3.506293,0,NaN,NaN,NaN,12,1
1,exp,16575,1382,3,NaN,1382.0,19625.0,3.0,images_meg/scoop/scoop_03s.jpg,4.70,4.706650,5.206864,0,NaN,NaN,NaN,12,1
2,test,22509,874,13,61.0,874.0,12520.0,14.0,images_test_meg/kimono_14s.jpg,6.00,6.257258,6.757498,0,NaN,NaN,NaN,12,1
3,exp,315,27,3,NaN,27.0,370.0,3.0,images_meg/antelope/antelope_03n.jpg,7.35,7.557743,8.057984,0,NaN,NaN,NaN,12,1
4,test,22535,1572,13,87.0,1572.0,22209.0,13.0,images_test_meg/streetlight_13s.jpg,8.90,8.908231,9.408491,0,NaN,NaN,NaN,12,1
5,exp,14775,1232,3,NaN,1232.0,17520.0,3.0,images_meg/potato/potato_03n.jpg,10.60,10.608922,11.109011,0,NaN,NaN,NaN,12,1
6,catch,22321,1855,73,NaN,NaN,NaN,NaN,images_catch_meg/catch073.jpg,12.15,12.159469,12.659736,1,82.0,14341.473888,0.435714,12,1
7,test,22517,1030,13,69.0,1030.0,14692.0,14.0,images_test_meg/mousetrap_14s.jpg,13.60,13.710108,14.210310,0,NaN,NaN,NaN,12,1
8,exp,8451,705,3,NaN,705.0,10108.0,3.0,images_meg/grapevine/grapevine_03s.jpg,15.15,15.160632,15.660787,0,NaN,NaN,NaN,12,1
9,exp,21903,1826,3,NaN,1826.0,25717.0,3.0,images_meg/wine_cooler/wine_cooler_03s.jpg,16.70,16.711247,17.211387,0,NaN,NaN,NaN,12,1


In [40]:
epoch.metadata.keys()

Index(['trial_type', 'image_nr', 'category_nr', 'exemplar_nr', 'test_image_nr',
       'things_category_nr', 'things_image_nr', 'things_exemplar_nr',
       'image_path', 'onset', 'image_on', 'image_off', 'responded', 'key_id',
       'key_time', 'RT', 'session_nr', 'run_nr'],
      dtype='object')

In [41]:
epoch.metadata['image_path']

0             images_meg/chain/chain_03s.jpg
1             images_meg/scoop/scoop_03s.jpg
2             images_test_meg/kimono_14s.jpg
3       images_meg/antelope/antelope_03n.jpg
4        images_test_meg/streetlight_13s.jpg
                        ...                 
2249    images_meg/anteater/anteater_03s.jpg
2250          images_meg/blimp/blimp_03n.jpg
2251              images_meg/rug/rug_03s.jpg
2252    images_meg/computer/computer_03n.jpg
2253    images_meg/coverall/coverall_03s.jpg
Name: image_path, Length: 2254, dtype: object

In [44]:
epoch.metadata['session_nr']

0       12
1       12
2       12
3       12
4       12
        ..
2249    12
2250    12
2251    12
2252    12
2253    12
Name: session_nr, Length: 2254, dtype: int64

In [115]:
epoch.metadata['image_path'].to_list()[:10]

['images_meg/chain/chain_03s.jpg',
 'images_meg/scoop/scoop_03s.jpg',
 'images_meg/antelope/antelope_03n.jpg',
 'images_meg/potato/potato_03n.jpg',
 'images_meg/grapevine/grapevine_03s.jpg',
 'images_meg/wine_cooler/wine_cooler_03s.jpg',
 'images_meg/football_helmet/football_helmet_03s.jpg',
 'images_meg/pump/pump_03s.jpg',
 'images_meg/icepick/icepick_03s.jpg',
 'images_meg/punch1/punch1_03s.jpg']

In [61]:
epoch[0]

Number of events,1
Events,4353: 1
Time range,-0.100 – 1.300 s
Baseline,off


In [64]:
epoch.selection

array([   0,    1,    2, ..., 2251, 2252, 2253])

In [65]:
epoch.info

<Info | 15 non-empty values
 bads: []
 ch_names: MLC11-1609, MLC12-1609, MLC13-1609, MLC14-1609, MLC15-1609, ...
 chs: 271 Magnetometers
 ctf_head_t: CTF/4D/KIT head -> head transform
 custom_ref_applied: False
 dev_ctf_t: MEG device -> CTF/4D/KIT head transform
 dev_head_t: MEG device -> head transform
 dig: 3 items (3 Cardinal)
 experimenter: hebart
 highpass: 0.1 Hz
 hpi_results: 1 item (list)
 lowpass: 40.0 Hz
 meas_date: 2019-07-12 12:14:00 UTC
 meas_id: 4 items (dict)
 nchan: 271
 projs: []
 sfreq: 200.0 Hz
 subject_info: 1 item (dict)
>

In [68]:
epoch.info.keys()

dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])

In [69]:
epoch.info['nchan']

271

In [70]:
ignore_indices = epoch.metadata.query('trial_type=="test"').index.to_list() + epoch.metadata.query('trial_type=="catch"').index.to_list()
epoch_dr = epoch.drop(ignore_indices)

Dropped 400 epochs: 2, 4, 6, 7, 14, 23, 29, 30, 34, 38, 48, 55, 56, 66, 73, 77, 81, 82, 84, 87, 88, 106, 117, 118, 133, 137, 140, 143, 151, 153, 157, 165, 174, 181, 189, 192, 193, 199, 205, 220, 233, 242, 254, 261, 269, 274, 277, 280, 283, 288, 289, 292, 295, 309, 315, 318, 319, 322, 332, 335, 340, 344, 351, 369, 371, 377, 381, 387, 393, 394, 401, 404, 405, 413, 424, 437, 438, 439, 441, 447, 457, 460, 465, 468, 474, 487, 491, 502, 503, 509, 510, 512, 519, 523, 537, 548, 552, 553, 559, 561, 570, 572, 576, 584, 590, 593, 594, 599, 600, 613, 617, 620, 624, 643, 646, 648, 652, 656, 659, 669, 684, 686, 690, 692, 703, 710, 720, 721, 722, 729, 733, 738, 745, 747, 750, 752, 757, 762, 766, 776, 792, 798, 809, 816, 822, 823, 827, 836, 843, 844, 852, 855, 860, 872, 876, 881, 883, 889, 891, 902, 911, 920, 931, 939, 943, 945, 955, 956, 966, 967, 968, 976, 977, 980, 984, 999, 1003, 1011, 1014, 1021, 1025, 1026, 1029, 1034, 1037, 1039, 1042, 1043, 1045, 1053, 1062, 1066, 1071, 1074, 1091, 1099, 1100,

In [72]:
epoch_dr.metadata

,trial_type,image_nr,category_nr,exemplar_nr,test_image_nr,things_category_nr,things_image_nr,things_exemplar_nr,image_path,onset,image_on,image_off,responded,key_id,key_time,RT,session_nr,run_nr
0,exp,3579,299,3,NaN,299.0,4353.0,3.0,images_meg/chain/chain_03s.jpg,3.00,3.006116,3.506293,0,NaN,NaN,NaN,12,1
1,exp,16575,1382,3,NaN,1382.0,19625.0,3.0,images_meg/scoop/scoop_03s.jpg,4.70,4.706650,5.206864,0,NaN,NaN,NaN,12,1
3,exp,315,27,3,NaN,27.0,370.0,3.0,images_meg/antelope/antelope_03n.jpg,7.35,7.557743,8.057984,0,NaN,NaN,NaN,12,1
5,exp,14775,1232,3,NaN,1232.0,17520.0,3.0,images_meg/potato/potato_03n.jpg,10.60,10.608922,11.109011,0,NaN,NaN,NaN,12,1
8,exp,8451,705,3,NaN,705.0,10108.0,3.0,images_meg/grapevine/grapevine_03s.jpg,15.15,15.160632,15.660787,0,NaN,NaN,NaN,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249,exp,303,26,3,NaN,26.0,358.0,3.0,images_meg/anteater/anteater_03s.jpg,333.25,333.331516,333.831749,0,NaN,NaN,NaN,12,10
2250,exp,1671,140,3,NaN,140.0,2131.0,3.0,images_meg/blimp/blimp_03n.jpg,334.65,334.832104,335.332262,0,NaN,NaN,NaN,12,10
2251,exp,16083,1341,3,NaN,1341.0,19070.0,3.0,images_meg/rug/rug_03s.jpg,336.00,336.215922,336.716176,0,NaN,NaN,NaN,12,10
2252,exp,4731,395,3,NaN,395.0,5680.0,3.0,images_meg/computer/computer_03n.jpg,337.60,337.599834,338.100026,0,NaN,NaN,NaN,12,10


In [73]:
dir(epochs)

['__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotations',
 '_bad_dropped',
 '_channel_type_idx',
 '_check_consistency',
 '_check_metadata',
 '_compute_aggregate',
 '_current',
 '_data',
 '_decim',
 '_decim_slice',
 '_detrend_offset_decim',
 '_detrend_picks',
 '_do_baseline',
 '_do_delayed_proj',
 '_evoked_from_epoch_data',
 '_filename',
 '_get_channel_positions',
 '_get_data',
 '_get_epoch_from_raw',
 '_get_name',
 '_getitem',
 '_handle_tmin_tmax',
 '_is_good_epoch',
 '_item_to_select',
 '_keys_to_idx',
 '_metadata',
 '_name',
 '_offset',
 '_pick_drop_channels',
 '_pick_projs'

In [83]:
dir(session_epochs[0])


['__class__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotations',
 '_bad_dropped',
 '_channel_type_idx',
 '_check_consistency',
 '_check_metadata',
 '_compute_aggregate',
 '_current',
 '_data',
 '_decim',
 '_decim_slice',
 '_detrend_offset_decim',
 '_detrend_picks',
 '_do_baseline',
 '_do_delayed_proj',
 '_evoked_from_epoch_data',
 '_filename',
 '_get_channel_positions',
 '_get_data',
 '_get_epoch_from_raw',
 '_get_name',
 '_getitem',
 '_handle_tmin_tmax',
 '_is_good_epoch',
 '_item_to_select',
 '_keys_to_idx',
 '_metadata',
 '_name',
 '_offset',
 '_pick_drop_channels',
 '_pick_projs'

In [84]:
session_epochs[0].load_data()

<EpochsArray |  2254 events (all good), -0.1 – 1.3 s, baseline off, ~1.28 GB, data loaded, with metadata,
 '341': 1
 '483': 1
 '805': 1
 '1014': 1
 '1042': 1
 '1329': 1
 '1704': 1
 '1794': 1
 '1807': 1
 '1835': 1
 and 2045 more events ...>

In [86]:
epochs = session_epochs[0]
epochs = epochs.load_data()
epochs.data

AttributeError: 'EpochsArray' object has no attribute 'data'

In [96]:
run_paths, event_paths = setup_paths(meg_dir, 1)
epochs = []
for run, curr_path in enumerate(run_paths):
    print('run: ', run)
    raw = read_raw(curr_path,session,run, participant)
    events = read_events(event_paths,run,raw)
    raw.filter(l_freq=l_freq,h_freq=h_freq)
    epochs = concat_epochs(raw, events, epochs)
    epochs.drop_bad()

run:  0
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.92   77.00    0.00 mm <->   -0.92   77.00    0.00 mm (orig :  -40.06   68.57 -237.90 mm) diff =    0.000 mm
       0.92  -77.00    0.00 mm <->    0.92  -77.00    0.00 mm (orig :   70.03  -39.14 -237.69 mm) diff =    0.000 mm
      79.58    0.00    0.00 mm <->   79.58   -0.00    0.00 mm (orig :   71.23   70.87 -233.83 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-01_meg.ds/sub-BIGMEG1_ses-01_task-main_run-01_meg.meg4: 
    System clock channel is available, checki

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.4s finished


Not setting metadata
226 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 226 events and 1681 original time points ...
0 bad epochs dropped
run:  1
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.71   77.07    0.00 mm <->   -0.71   77.07    0.00 mm (orig :  -40.27   68.70 -237.97 mm) diff =    0.000 mm
       0.71  -77.07    0.00 mm <->    0.71  -77.07    0.00 mm (orig :   69.91  -39.09 -237.96 mm) diff =    0.000 mm
      79.46    0.00    0.00 mm <->   79.46   -0.00    0.00 mm (orig :   70.83   71.02 -233.95 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device c

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.2s finished


Not setting metadata
226 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 226 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 226 events and 1681 original time points ...
Using data from preloaded Raw for 226 events and 1681 original time points ...
Not setting metadata
452 matching events found
No baseline correction applied
run:  2
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-03_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.64   77.10    0.00 mm <->   -0.64   77.10   -0.00 mm (orig :  -40.41   68.84 -238.00 mm) diff =    0.000 mm
       0.64  -77.10    0.00 mm <->    0.64  -77.10    0.00 mm (orig :   69.73  -39.08 -238.46 

...

Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
1804 matching events found
No baseline correction applied
run:  8
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-09_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.84   77.17    0.00 mm <->   -0.84   77.17    0.00 mm (orig :  -40.16   68.71 -237.52 mm) diff =    0.000 mm
       0.84  -77.17    0.00 mm <->    0.84  -77.17    0.00 mm (orig :   70.00  -39.40 -238.12 mm) diff =    0.000 mm
      79.17    0.00    0.00 mm <->   79.17   -0.00    0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.2s finished


Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
2029 matching events found
No baseline correction applied
run:  9
ds directory : /work/project/MEG_GOD/yainoue/things/MEG/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-10_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.97   77.23    0.00 mm <->   -0.97   77.23   -0.00 mm (orig :  -40.35   68.65 -237.96 mm) diff =    0.000 mm
       0.97  -77.23    0.00 mm <->    0.97  -77.23   -0.00 mm (orig :   70.03  -39.41 -238.18 mm) diff =    0.000 mm
      79.23    0.00    0.00 mm <->   79.23    0.00    0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    9.2s finished


Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
2254 matching events found
No baseline correction applied


In [123]:
 epoch = mne.Epochs(raw, events, tmin = pre_stim_time, tmax = post_stim_time, picks = 'mag',baseline=None)

Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated


In [124]:
epoch2 = mne.Epochs(raw, events, tmin = pre_stim_time, tmax = post_stim_time, picks = 'meg',baseline=None)

Not setting metadata
225 matching events found
No baseline correction applied
0 projection items activated


In [134]:
data = raw.get_data()

In [135]:
data.shape

(309, 417600)

In [136]:
epoch.get_data().shape

Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped


(225, 271, 1681)

In [143]:
# epochs = session_epochs[0]
data = epochs.get_data()
data.shape

(2254, 271, 1681)

In [144]:
epochs.metadata